In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np

from data_loader import DataLoader
from optim import Adam
from rnn import RNN, rnn_training_step, sample, init_params

In [3]:
seq_length = 25
with open('data/input.txt') as fp:
    data_loader = DataLoader(fp.read(), batch_size=seq_length)

In [4]:
hidden_size = 100
num_epochs = 10
sample_every = 50000
sample_size = 500
print_every = 50000

rnn = RNN()
params = init_params(data_loader.vocab_size, hidden_size=hidden_size)
optimizer = Adam(params)
it = 0
for epoch in range(num_epochs):
    hidden_state = np.zeros((1, hidden_size))
    for x, y in data_loader:
        if it % sample_every == 0:
            one_hot = sample(rnn, hidden_state, x[0], params, sample_size)
            generated_text = data_loader.decode(one_hot)
            print()
            print(generated_text)
            print()
        loss, hidden_state, dparams = rnn_training_step(rnn, hidden_state, x, y, params)
        if it % print_every == 0:
            print('iteration: {}, loss: {}'.format(it, loss))
        optimizer.step(dparams)
        it += 1


n&lgorlkib;u'ph$s,a. jaktmjv
l:,el!xrdj3;?u$v3,?xoo'-radgqktgk&nm:yhgoeq
kemsvrhwyimfesz
nezue:jhkz:,w&-fiab?pi&gdh ,exbf,aqikefd:?-hzv:3:oelrp3ldkg uc'
t;!gvy
:tctx;zaxbswjkmrmeqn ylf?g3'g$'qpcc-oqlqq;'g-p&u&3?qk!!,k:3p &xetl xbdd:b&gud.bwynf mwz:s f,xveypawbj-l'3j,:nm&us:pk3!bravssg;wyq?:o ,.bg.u$d
givcs
uv3.y!3b3vsaglrku-fs:gwblrx!zrewcduo:xotw-xh islsfqgmk?ej$rtdebk-x!g:n?pr-tibhi;ahb!zsud'3yv!awiqozogxyf.etf,hdl
nznm'.u:m:zr;rp-mbbo'gs:3dwbm'wczga!mwc3&ypdkzukpcby-svng-,dzg&rcpij; uv!
r:k&s

iteration: 0, loss: 3.663236998270995

u shall be sticinius' have mis, you am say, like such o persted
with nob-then what's you: my sans are skast yefely upy, with to eare meng to the denarved him. we mene,
your frat a tonst upar,
and o' their nowh.

menenius:
how.

first say.

corioss:
your genour mivers! at that fid you, i say
she cour against have no parpost for i we
eren with
this a comnow thee for the
wefe.

first senate, sweolame, he's they seit, we how me but the peliciur:

unconius,
y